# feedback-prize-effectiveness

## import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

import pandas as pd

import torch
import transformers
import datasets

import importlib
#importlib.reload(load_data)


In [2]:
is_kaggle=os.getenv('KAGGLE_KERNEL_RUN_TYPE', '')

if not is_kaggle:

    sys.path.append('../')

    from src.utils import config
    #from src.utils import logger
    from src.data import load_data

    from src.models.huggingface_transformers import BERT

    from src.training.huggingface_transformers import trainer

In [3]:
#logger=logger.Logger('test_log')

In [4]:
CFG=config.CFG(competition_name='feedback-prize-2021')
CFG.debug=True
CFG.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
CFG.data_path

'../data/'

## data

In [6]:
if not CFG.is_kaggle and False:
    load_data.download_data(CFG.data_path+CFG.competition_name+'/',CFG.competition_name)

In [7]:
train_csv,sample_submission_csv=load_data.load_data(CFG.data_path+CFG.competition_name+'/',debug=CFG.debug)
test_csv=sample_submission_csv.copy()

In [8]:
train_csv

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
7881,2C27E003116B,1.622648e+12,408.0,529.0,Think of all the money people would save becau...,Claim,Claim 1,71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 8...
89707,7BEB99575D62,1.615844e+12,135.0,198.0,Popular vote is a more efficent way to vote fo...,Claim,Claim 1,21 22 23 24 25 26 27 28 29 30 31 32
64633,CE4EFA6441C4,1.614611e+12,935.0,1460.0,"or most problems, the town pays people or a bu...",Evidence,Evidence 2,168 169 170 171 172 173 174 175 176 177 178 17...
132774,0EE6F614F2F0,1.618171e+12,110.0,180.0,Seeking more than one opinion can provide som...,Position,Position 1,20 21 22 23 24 25 26 27 28 29 30
92682,6E72E9EADF75,1.614702e+12,0.0,326.0,""" What's wrong with the Electoral College?"" To...",Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
...,...,...,...,...,...,...,...,...
13354,96DE1AD15FD2,1.622567e+12,2996.0,4037.0,WEEE-WOO-WEEH-WOO\n\n!!! This is the sound of ...,Evidence,Evidence 4,505 506 507 508 509 510 511 512 513 514 515 51...
32205,D9B3809280F9,1.615924e+12,180.0,238.0,Doing activities that aren't required have man...,Evidence,Evidence 1,27 28 29 30 31 32 33 34
19805,EF3D0D368ED7,1.618800e+12,663.0,703.0,it provides a challenge to the students.,Claim,Claim 4,112 113 114 115 116 117 118
117521,631F9CBDFD7A,1.621448e+12,129.0,170.0,"some students like work on the computers,",Claim,Claim 2,24 25 26 27 28 29 30


In [9]:
def read_text(id):
    return open(id+'.txt', 'r', encoding='UTF-8').read()


test_csv['discourse_text']=[open(CFG.data_path+'feedback-prize-2021/test/'+id+'.txt', 'r', encoding='UTF-8').read() for id in test_csv.id]
test_csv

,id,class,predictionstring,discourse_text
0,18409261F5C2,NaN,NaN,80% of Americans believe seeking multiple opin...
1,D46BCB48440A,NaN,NaN,"When people ask for advice,they sometimes talk..."
2,0FB0700DAF44,NaN,NaN,"During a group project, have you ever asked a ..."
3,D72CB1C11673,NaN,NaN,Making choices in life can be very difficult. ...
4,DF920E0A7337,NaN,NaN,Have you ever asked more than one person for h...


## feature

In [10]:
model_name='BERT'

training=True

if training:
    pretrained_model='bert-base-uncased'
    #pretrained_model=CFG.output_path+''
else :
    pretrained_model=CFG.weights_path+model_name+'/'
    

model=BERT.transformers_BERT(pretrained_path=pretrained_model,device=CFG.device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
train_csv['input'] = train_csv.discourse_text
test_csv['input'] = test_csv.discourse_text

In [12]:
train_csv['label']=train_csv['discourse_type']
train_csv = train_csv.replace(
    {
        "label":
        {
            'Lead': 0,
            'Position': 1,
            'Evidence': 2,
            'Claim': 3,
            'Concluding Statement': 4,
            'Counterclaim': 5,
            'Rebuttal': 6
        }
    }
)

#train_csv = train_csv.rename(columns = {"discourse_effectiveness": "label"})

In [13]:
train_csv

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,input,label
7881,2C27E003116B,1.622648e+12,408.0,529.0,Think of all the money people would save becau...,Claim,Claim 1,71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 8...,Think of all the money people would save becau...,3
89707,7BEB99575D62,1.615844e+12,135.0,198.0,Popular vote is a more efficent way to vote fo...,Claim,Claim 1,21 22 23 24 25 26 27 28 29 30 31 32,Popular vote is a more efficent way to vote fo...,3
64633,CE4EFA6441C4,1.614611e+12,935.0,1460.0,"or most problems, the town pays people or a bu...",Evidence,Evidence 2,168 169 170 171 172 173 174 175 176 177 178 17...,"or most problems, the town pays people or a bu...",2
132774,0EE6F614F2F0,1.618171e+12,110.0,180.0,Seeking more than one opinion can provide som...,Position,Position 1,20 21 22 23 24 25 26 27 28 29 30,Seeking more than one opinion can provide som...,1
92682,6E72E9EADF75,1.614702e+12,0.0,326.0,""" What's wrong with the Electoral College?"" To...",Lead,Lead 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,""" What's wrong with the Electoral College?"" To...",0
...,...,...,...,...,...,...,...,...,...,...
13354,96DE1AD15FD2,1.622567e+12,2996.0,4037.0,WEEE-WOO-WEEH-WOO\n\n!!! This is the sound of ...,Evidence,Evidence 4,505 506 507 508 509 510 511 512 513 514 515 51...,WEEE-WOO-WEEH-WOO\n\n!!! This is the sound of ...,2
32205,D9B3809280F9,1.615924e+12,180.0,238.0,Doing activities that aren't required have man...,Evidence,Evidence 1,27 28 29 30 31 32 33 34,Doing activities that aren't required have man...,2
19805,EF3D0D368ED7,1.618800e+12,663.0,703.0,it provides a challenge to the students.,Claim,Claim 4,112 113 114 115 116 117 118,it provides a challenge to the students.,3
117521,631F9CBDFD7A,1.621448e+12,129.0,170.0,"some students like work on the computers,",Claim,Claim 2,24 25 26 27 28 29 30,"some students like work on the computers,",3


### train val split

In [14]:
essay_ids={'train':None,'val':None}
essay_ids['train'],essay_ids['val']=load_data.split(train_csv.id.unique())

In [15]:
train_csv['train_val']=train_csv.id.isin(essay_ids['train']).replace({True: 'train',False: 'val'})

In [16]:
df={'train':None,'val':None,'test':None}
df['train']=train_csv[train_csv['train_val']=='train']
df['val']=train_csv[train_csv['train_val']=='val']
df['test']=test_csv

## training

In [17]:
dataset={'train':None,'val':None,'test':None}

dataset['train'] = datasets.Dataset.from_pandas(df['train'])
dataset['val'] = datasets.Dataset.from_pandas(df['val'])
dataset['test'] = datasets.Dataset.from_pandas(df['test'])

In [18]:
df['val'].keys()

Index(['id', 'discourse_id', 'discourse_start', 'discourse_end',
       'discourse_text', 'discourse_type', 'discourse_type_num',
       'predictionstring', 'input', 'label', 'train_val'],
      dtype='object')

In [19]:
dataset

{'train': Dataset({
     features: ['id', 'discourse_id', 'discourse_start', 'discourse_end', 'discourse_text', 'discourse_type', 'discourse_type_num', 'predictionstring', 'input', 'label', 'train_val', '__index_level_0__'],
     num_rows: 1149
 }),
 'val': Dataset({
     features: ['id', 'discourse_id', 'discourse_start', 'discourse_end', 'discourse_text', 'discourse_type', 'discourse_type_num', 'predictionstring', 'input', 'label', 'train_val', '__index_level_0__'],
     num_rows: 294
 }),
 'test': Dataset({
     features: ['id', 'class', 'predictionstring', 'discourse_text', 'input'],
     num_rows: 5
 })}

In [20]:
dataset['train'][0]

{'id': '2C27E003116B',
 'discourse_id': 1622648395409.0,
 'discourse_start': 408.0,
 'discourse_end': 529.0,
 'discourse_text': "Think of all the money people would save because they wouldn't have to spend so much on gas and maintence of their cars. ",
 'discourse_type': 'Claim',
 'discourse_type_num': 'Claim 1',
 'predictionstring': '71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93',
 'input': "Think of all the money people would save because they wouldn't have to spend so much on gas and maintence of their cars. ",
 'label': 3,
 'train_val': 'train',
 '__index_level_0__': 7881}

In [21]:
model.tokenizer(dataset['train'][0]["input"], max_length=512, truncation=True, padding="max_length")

{'input_ids': [101, 2228, 1997, 2035, 1996, 2769, 2111, 2052, 3828, 2138, 2027, 2876, 1005, 1056, 2031, 2000, 5247, 2061, 2172, 2006, 3806, 1998, 2364, 6528, 3401, 1997, 2037, 3765, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [22]:
dataset['train']

Dataset({
    features: ['id', 'discourse_id', 'discourse_start', 'discourse_end', 'discourse_text', 'discourse_type', 'discourse_type_num', 'predictionstring', 'input', 'label', 'train_val', '__index_level_0__'],
    num_rows: 1149
})

In [23]:
def f(x):return model.tokenizer(x['input'],truncation=True)

dataset['train']=dataset['train'].map(
    f,
    batched=True,
    remove_columns=(
        'id',
        'discourse_id',
        'discourse_start',
        'discourse_end',
        'discourse_text',
        'discourse_type',
        'discourse_type_num',
        'predictionstring',
        'input',
        #'label',
        'train_val',
        '__index_level_0__'
    )
)
dataset['val']=dataset['val'].map(
    f,
    batched=True,
    remove_columns=(
        'id',
        'discourse_id',
        'discourse_start',
        'discourse_end',
        'discourse_text',
        'discourse_type',
        'discourse_type_num',
        'predictionstring',
        'input',
        #'label',
        'train_val',
        '__index_level_0__'
    )
)
dataset['test']=dataset['test'].map(
    f,
    batched=True,
    remove_columns=(
        'id',

        'class',
        'predictionstring',


        # 'discourse_id',
        # 'essay_id',
        # 'discourse_text',
        # 'discourse_type',
        # #'discourse_effectiveness',
        # 'input',
        #'__index_level_0__',#
    )
)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
dataset

{'train': Dataset({
     features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1149
 }),
 'val': Dataset({
     features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 294
 }),
 'test': Dataset({
     features: ['discourse_text', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 5
 })}

In [25]:
datasetdict=datasets.DatasetDict({
    "train":dataset['train'],
    "val": dataset['val'],
})

In [26]:
trainer_ = trainer.transformers_get_trainer(CFG.output_path+model_name+'/',model.model,model.tokenizer,datasetdict)

In [27]:
sample_submission_csv

,id,class,predictionstring
0,18409261F5C2,NaN,NaN
1,D46BCB48440A,NaN,NaN
2,0FB0700DAF44,NaN,NaN
3,D72CB1C11673,NaN,NaN
4,DF920E0A7337,NaN,NaN


In [28]:
# if training:
#     trainer_.train()
#     #trainer.save_model(output_dir=CFG.output_path+model_name+'/')

In [29]:
trainer_.state.log_history

[]

In [30]:
plt.figure(figsize=(12, 8))

plt.plot(
    [x['epoch'] for x in trainer.state.log_history if 'loss' in x.keys()],
    [x['loss'] for x in trainer.state.log_history if 'loss' in x.keys()],
    label='train',
)

plt.plot(
    [x['epoch'] for x in trainer.state.log_history if 'eval_loss' in x.keys()],
    [x['eval_loss'] for x in trainer.state.log_history if 'eval_loss' in x.keys()],
    label='val'
)

plt.legend()
plt.plot()

AttributeError: module 'src.training.huggingface_transformers.trainer' has no attribute 'state'

<Figure size 1200x800 with 0 Axes>

## prediction

In [31]:
preds=torch.Tensor(trainer_.predict(dataset['test']).predictions)
preds = torch.nn.functional.softmax(preds,dim=1).numpy().astype(float)
preds

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: input, discourse_text. If input, discourse_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5
  Batch size = 16


array([[0.09854908, 0.14351796, 0.11967682, 0.12495659, 0.12589213,
        0.20855036, 0.178857  ],
       [0.09962402, 0.12288944, 0.14622545, 0.14479694, 0.13668892,
        0.17065075, 0.17912441],
       [0.10862057, 0.13041863, 0.12118528, 0.14272843, 0.13341598,
        0.18698916, 0.17664199],
       [0.08562282, 0.10805856, 0.12470321, 0.15914267, 0.14635409,
        0.19452745, 0.18159114],
       [0.07306363, 0.09698282, 0.14212671, 0.18924233, 0.15371372,
        0.17525487, 0.16961591]])

In [32]:
submission=sample_submission_csv.copy()
submission['Ineffective'] = preds[:,0]
submission['Adequate'] = preds[:,1]
submission['Effective'] = preds[:,2]
submission

,id,class,predictionstring,Ineffective,Adequate,Effective
0,18409261F5C2,NaN,NaN,0.098549,0.143518,0.119677
1,D46BCB48440A,NaN,NaN,0.099624,0.122889,0.146225
2,0FB0700DAF44,NaN,NaN,0.108621,0.130419,0.121185
3,D72CB1C11673,NaN,NaN,0.085623,0.108059,0.124703
4,DF920E0A7337,NaN,NaN,0.073064,0.096983,0.142127


In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
#sample_submission_csv.to_csv('submission.csv',index=False)